In [1]:
from library.objective_function import *
from library.optimiser import *
from library.experiments import *
from library.post_analysis import *
from library.cfd_objective import *

### 1 check mesh shape of cars latents

In [1]:
def load_pressure_predictor(load_directory):
    model = SplineCNN8Residuals(3)
    model.load_state_dict(torch.load(load_directory + "/cfdModel.nn"))
    model = model.to("cuda:0").eval()
    return model

def load_latent_vectors(load_directory, checkpoint):
    filename = os.path.join(
        load_directory, checkpoint + ".pth"
    )
    if not os.path.isfile(filename):
        raise Exception(
            "The experiment directory ({}) does not include a latent code file"
            + " for checkpoint '{}'".format(load_directory, checkpoint)
        )
    data = torch.load(filename)
    return data["latent_codes"].cuda()

def load_decoder(load_directory, checkpoint):
    specs_filename = os.path.join(load_directory, "specs.json")
    if not os.path.isfile(specs_filename):
        raise Exception(
            'The experiment directory does not include specifications file "specs.json"'
        )
    specs = json.load(open(specs_filename))
    latent_size = specs["CodeLength"]
    decoder = Decoder(latent_size, **specs["NetworkSpecs"])
    decoder = torch.nn.DataParallel(decoder)
    saved_model_state = torch.load(os.path.join(load_directory, checkpoint +".pth"))
    decoder.load_state_dict(saved_model_state["model_state_dict"])
    decoder = decoder.module.cuda()
    decoder.eval()
    return decoder

In [2]:
DIR_to_load_data = 'starting_data'
experiment_directory = "data_for_this_experiments"

predictor = load_pressure_predictor(DIR_to_load_data)

decoder = load_decoder(DIR_to_load_data, "decoderModel")

latent_vectors = load_latent_vectors(DIR_to_load_data, "latentCodes").detach()

AvgTransform = np.load(DIR_to_load_data + "/avg_trans_matrix.npy") #computeAvgTransform()

LATENT_TO_OPTIMIZE = latent_vectors[32]
LATENT_KD_TREE = KDTree(np.array([lv.cpu().detach().numpy()[0] for lv in latent_vectors]))

NameError: name 'SplineCNN8Residuals' is not defined

In [4]:
torch.manual_seed(0)

# 1 prepare data
latent = latent_vectors[32]

# 2 prepare model
#decoder = load_decoder(DIR_to_load_data, "decoderModel")
#predictor = load_pressure_predictor(DIR_to_load_data)

# init and setup one experiment
exp = single_experiment()
# One experiment: setup objective function
objective = cfd_obj(decoder, predictor, latent_vectors, AvgTransform, LATENT_KD_TREE)
exp.set_objective(objective)
opt = adam(dim=256)
optParas = {
         'x0': latent,
         'alpha': 8e-3,
         'beta_1': 0.9, 
         'beta_2': 0.999, 
         'epsilon': 1e-11, 
         'max_iter': 10,
         'tol': 1e-6,              
         'verbose': True,
         'record': True}
opt.set_parameters(optParas)
exp.set_optimizer(opt)
%time stats_ada = exp.do()



*******starting optimisation from intitial point:  0.57972956
first part of loss:  -0.008534351
second part of loss:  0.0
third part of loss:  0.0
Drag_point:  0.32287428
Drag*normal:  0.19913064 backward loss:  0.00021854066
penalty:  0.079743706
loss + penalty:  0.079962246
norm of grad 0.39857736 

first part of loss:  -0.015950955
second part of loss:  0.0
third part of loss:  0.0
Drag_point:  0.33058915
Drag*normal:  0.20049322 backward loss:  -0.00024022127
penalty:  0.073850594
loss + penalty:  0.07361037
norm of grad 0.36500147 

first part of loss:  -0.020603903
second part of loss:  0.0
third part of loss:  0.0
Drag_point:  0.32830766
Drag*normal:  0.19876316 backward loss:  -4.1640382e-05
penalty:  0.07053756
loss + penalty:  0.07049592
norm of grad 0.34678316 

first part of loss:  -0.020075232
second part of loss:  0.0
third part of loss:  0.0
Drag_point:  0.325788
Drag*normal:  0.19493012 backward loss:  -0.00014247079
penalty:  0.068881445
loss + penalty:  0.068738975
